In [6]:
import pandas as pd 
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np
import pickle
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
import lightgbm as lgb
import matplotlib.pyplot as plt
from scipy.stats import randint as sp_randint
from sklearn.model_selection import  RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import  classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer, confusion_matrix, plot_confusion_matrix
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
import ppscore as pps
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
Data=pd.read_csv("../Data/Final_prepared_data.csv")

In [3]:
Data=Data.set_index("SK_ID_CURR")

In [4]:
Data.CODE_GENDER.replace({"XNA":np.nan},inplace=True)
Data.NAME_FAMILY_STATUS.replace({"Unknown":np.nan},inplace=True)
Data.ORGANIZATION_TYPE.replace({"XNA":np.nan},inplace=True)

In [9]:
Data.head(5)

,EXT_SRC_TOT,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_LAST_PHONE_CHANGE,HOUR_APPR_PROCESS_START,AMT_INCOME_TOTAL,AMT_ANNUITY,...,ORGANIZATION_TYPE,FLAG_OWN_REALTY,FLAG_WORK_PHONE,NAME_FAMILY_STATUS,NAME_TYPE_SUITE,DEF_60_CNT_SOCIAL_CIRCLE,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_WORK_CITY,CODE_GENDER,TARGET
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,0.402324,0.262949,0.139376,-9461,-2120,-3648.0,-1134.0,10,202500.0,24700.5,...,Business Entity Type 3,Y,0,Single / not married,Unaccompanied,2.0,2,0,M,1
100003,NaN,0.622246,NaN,-16765,-291,-1186.0,-828.0,11,270000.0,35698.5,...,School,N,0,Married,Family,0.0,1,0,F,0
100004,1.285479,0.555912,0.729567,-19046,-2531,-4260.0,-815.0,9,67500.0,6750.0,...,Government,Y,1,Single / not married,Unaccompanied,0.0,2,0,M,0
100006,NaN,0.650442,NaN,-19005,-2437,-9833.0,-617.0,17,135000.0,29686.5,...,Business Entity Type 3,Y,0,Civil marriage,Unaccompanied,0.0,2,0,F,0
100007,NaN,0.322738,NaN,-19932,-3458,-4311.0,-1106.0,11,121500.0,21865.5,...,Religion,Y,0,Single / not married,Unaccompanied,0.0,2,1,M,0


In [10]:
Data.shape

(307511, 31)

In [37]:
type(np.NaN)

float

# 1-Split_Data

In [11]:
y=Data.loc[:,'TARGET']
X=Data.drop('TARGET',axis=1)

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 2-Imputation

In [13]:
Categorical_features=[f for f in X_train.columns  if X_train[f].dtype=='O']
Numerical_features=[f for f in X_train.columns  if X_train[f].dtype!='O' and f!='TARGET']

In [14]:
cat_imputer = SimpleImputer( missing_values=np.nan,strategy='most_frequent').fit(X_train[Categorical_features])
num_imputer = SimpleImputer( missing_values=np.nan,strategy='median').fit(X_train[Numerical_features])

In [11]:
def impute_features(cat_imputer,num_imputer,train=True):
    if train:
            X_train[Categorical_features]=cat_imputer.transform(X_train[Categorical_features])
            X_train[Numerical_features]=num_imputer.transform(X_train[Numerical_features])
    else: 
            X_valid[Categorical_features]=cat_imputer.transform(X_valid[Categorical_features])
            X_valid[Numerical_features]=num_imputer.transform(X_valid[Numerical_features])

In [12]:
impute_features(cat_imputer,num_imputer)
impute_features(cat_imputer,num_imputer,False)

# 3-Ordinal_Encoding

We decided to go with ordinal encoding rather than ohe to reduce sparesity of data as it hurts

the performance in case of tree based model used  

check this: https://stackoverflow.com/questions/65722374/one-hot-encoding-in-random-forest-classifier

In [15]:
OEnc=OrdinalEncoder(handle_unknown='skip')

In [14]:
X_train[Categorical_features]=OEnc.fit_transform(X_train[Categorical_features])
X_valid[Categorical_features]=OEnc.transform(X_valid[Categorical_features])

# 4-Model

We will go with lightGBM as it is :

1-Ensemble algorithm tree based

2-Can adjust weights during training to handel class imbalance

In [15]:
clf = lgb.LGBMClassifier(max_depth=10,min_data_in_leaf=45,learning_rate= 0.01,n_estimators= 500,is_unbalance = True)

In [30]:
clf.fit(X_train,y_train)

[LightGBM] [Warning] min_data_in_leaf is set=45, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=45


LGBMClassifier(is_unbalance=True, learning_rate=0.01, max_depth=10,
               min_data_in_leaf=45, n_estimators=500)

In [31]:

prob_preds=clf.predict_proba(X_valid)
print('AUC: ', roc_auc_score(y_valid, prob_preds[:,1]))

AUC:  0.7542875084023197


# 5-Hyperparameters tuning

Handeling class imbalance by scale_pos_weight instead of is_unbalanced :

https://www.kaggle.com/code/mlisovyi/modular-good-fun-with-ligthgbm/comments

In [28]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]

num_leaves=[int(x) for x in np.linspace(start = 5 ,stop = 50, num = 5)]

max_depth = [int(x) for x in np.linspace(5, 20, num = 2)]

scale_pos_weight=[int(x) for x in np.linspace(start = 1, stop = 11, num = 11)]

random_grid = {'n_estimators': n_estimators,
               'num_leaves': num_leaves,
               'max_depth': max_depth,
               'scale_pos_weight': scale_pos_weight}


lgbm_random = RandomizedSearchCV(estimator=LGBMClassifier(n_jobs=-1), param_distributions=random_grid, cv = 3, n_iter=10,return_train_score=True,scoring='roc_auc')

lgbm_random.fit(X_train, y_train)

RandomizedSearchCV(cv=3, estimator=LGBMClassifier(),
                   param_distributions={'max_depth': [5, 20],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000],
                                        'num_leaves': [5, 16, 27, 38, 50],
                                        'scale_pos_weight': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11]},
                   return_train_score=True, scoring='roc_auc')

In [17]:
best_lgbm=LGBMClassifier(max_depth=5, n_estimators=1000, num_leaves=5,
                                scale_pos_weight=5,random_state=22)

# Full_Pipeline

In [18]:
categorical_transformer = Pipeline(
    steps=[
        ('catimputer', cat_imputer),
        ('ordinal', OEnc)])


numeric_transformer = num_imputer

In [19]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric_transformers', numeric_transformer, Numerical_features),
        ('categorical_transformers', categorical_transformer, Categorical_features),
    ])

In [20]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', best_lgbm)])
pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_transformers',
                                                  SimpleImputer(strategy='median'),
                                                  ['EXT_SRC_TOT',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', 'DAYS_BIRTH',
                                                   'DAYS_ID_PUBLISH',
                                                   'DAYS_REGISTRATION',
                                                   'DAYS_LAST_PHONE_CHANGE',
                                                   'HOUR_APPR_PROCESS_START',
                                                   'AMT_INCOME_TOTAL',
                                                   'AMT_ANNUITY',
                                                   'REGION_POPULATION_RELATIVE',
                                                   'NEW_CREDIT_TO_ANNUI...
     

In [21]:
predictions = pipeline.predict_proba(X_valid)
print('AUC: ', roc_auc_score(y_valid, predictions[:,1]))

AUC:  0.7615475864391146


# Function for inference

In [24]:
def predict_datapoint(model_pipeline,test_point):
    features_names=['EXT_SRC_TOT','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH','DAYS_ID_PUBLISH','DAYS_REGISTRATION','DAYS_LAST_PHONE_CHANGE','HOUR_APPR_PROCESS_START','AMT_INCOME_TOTAL','AMT_ANNUITY','REGION_POPULATION_RELATIVE','NEW_CREDIT_TO_ANNUITY_RATIO','NEW_CREDIT_TO_GOODS_RATIO','AMT_REQ_CREDIT_BUREAU_YEAR','OBS_30_CNT_SOCIAL_CIRCLE','CNT_CHILDREN','AMT_REQ_CREDIT_BUREAU_QRT','FLAG_PHONE','AMT_REQ_CREDIT_BUREAU_MON','WEEKDAY_APPR_PROCESS_START','DEF_30_CNT_SOCIAL_CIRCLE','ORGANIZATION_TYPE','FLAG_OWN_REALTY','FLAG_WORK_PHONE','NAME_FAMILY_STATUS','NAME_TYPE_SUITE','DEF_60_CNT_SOCIAL_CIRCLE','REGION_RATING_CLIENT_W_CITY','REG_CITY_NOT_WORK_CITY','CODE_GENDER']
    x=pd.DataFrame(np.expand_dims(test_point,0),columns=features_names)
    Non_defaultor_prob,defaultor_prob=model_pipeline.predict_proba(x)[0]
    print("Non defaultor probability is {}%".format(Non_defaultor_prob*100))
    print("=======================================================")
    print('Defaultor probability is {}%'.format(defaultor_prob*100))
    
    
    

In [29]:
rand_idx=np.random.choice(100,1)
rand_point=X_valid.iloc[rand_idx,:]
predict_datapoint(pipeline,rand_point)

Non defaultor probability is 90.01099004011411%
Defaultor probability is 9.989009959885891%


Save Pipeline

In [31]:
pickle.dump(pipeline,open("lgbm_pipeline.pkl",'wb'))